In [ ]:
https://www.flashscoreusa.com/soccer/england/premier-league-2022-2023/results/

In [3]:
import pandas as pd
import time
from pathlib import Path
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import warnings
import pandas as pd
import sys
from pathlib import Path
warnings.filterwarnings("ignore")


In [4]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])

# Add user agent to avoid bot detection
chrome_options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

# Performance optimizations
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--disable-extensions')


driver = webdriver.Chrome(options=chrome_options)
print("Chrome WebDriver initialized")

Chrome WebDriver initialized


In [6]:
url = "https://www.flashscoreusa.com/soccer/england/premier-league-2022-2023/results/"

In [ ]:
# All the urls
"https://www.flashscoreusa.com/soccer/england/premier-league-2022-2023/results/"
"https://www.flashscoreusa.com/soccer/italy/serie-a-2022-2023/results/"
"https://www.flashscoreusa.com/soccer/spain/laliga-2022-2023/results/"
"https://www.flashscoreusa.com/soccer/germany/bundesliga-2022-2023/results/"
"https://www.flashscoreusa.com/soccer/europe/champions-league-2022-2023/results/"
"https://www.flashscoreusa.com/soccer/spain/laliga2-2022-2023/results/"
"https://www.flashscore.com/football/netherlands/eredivisie-2022-2023/results/"
"https://www.flashscore.com/football/belgium/jupiler-pro-league-2022-2023/results/"

"https://www.flashscoreusa.com/soccer/china/super-league-2023/results/"
"https://www.flashscoreusa.com/soccer/south-korea/k-league-1-2023/results/"


In [9]:
import pandas as pd
import time
from pathlib import Path
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import warnings
import pandas as pd
import sys
from pathlib import Path
warnings.filterwarnings("ignore")

# Define all league URLs with their corresponding league names
league_urls = {
    "Premier League 2022-2023": "https://www.flashscoreusa.com/soccer/england/premier-league-2022-2023/results/",
    "Serie A 2022-2023": "https://www.flashscoreusa.com/soccer/italy/serie-a-2022-2023/results/",
    "La Liga 2022-2023": "https://www.flashscoreusa.com/soccer/spain/laliga-2022-2023/results/",
    "Bundesliga 2022-2023": "https://www.flashscoreusa.com/soccer/germany/bundesliga-2022-2023/results/",
    "Champions League 2022-2023": "https://www.flashscoreusa.com/soccer/europe/champions-league-2022-2023/results/",
    "La Liga 2 2022-2023": "https://www.flashscoreusa.com/soccer/spain/laliga2-2022-2023/results/",
    "Eredivisie 2022-2023": "https://www.flashscore.com/football/netherlands/eredivisie-2022-2023/results/",
    "Jupiler Pro League 2022-2023": "https://www.flashscore.com/football/belgium/jupiler-pro-league-2022-2023/results/",
    "Chinese Super League 2023": "https://www.flashscoreusa.com/soccer/china/super-league-2023/results/",
    "K-League 1 2023": "https://www.flashscoreusa.com/soccer/south-korea/k-league-1-2023/results/"
}

def initialize_driver():
    """Initialize Chrome WebDriver with optimized settings"""
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
    
    # Add user agent to avoid bot detection
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    
    # Performance optimizations
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--disable-extensions')
    
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def load_all_games(driver, max_clicks=100):
    """Load all games by clicking 'Show more games' button"""
    clicks = 0
    while clicks < max_clicks:
        try:
            # Look for the "Show more games" button
            show_more_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "span[data-testid='wcl-scores-caption-05']"))
            )
            
            # Scroll to button and click
            driver.execute_script("arguments[0].scrollIntoView();", show_more_btn)
            time.sleep(1)
            show_more_btn.click()
            
            print(f"  → Clicked 'Show more games' - Click #{clicks + 1}")
            time.sleep(2)  # Wait for new content to load
            clicks += 1
            
        except TimeoutException:
            print("  ✓ No more 'Show more games' button found")
            break
        except Exception as e:
            print(f"  ✗ Error clicking button: {e}")
            break
    
    return clicks

def scrape_league_matches(driver, url, league_name):
    """Scrape matches from a specific league URL"""
    print(f"\n{'='*60}")
    print(f"Scraping: {league_name}")
    print(f"URL: {url}")
    print(f"{'='*60}")
    
    try:
        driver.get(url)
        time.sleep(3)
        
        # Load all games
        total_clicks = load_all_games(driver)
        print(f"  ✓ Loaded all games after {total_clicks} clicks")
        
        # Get page source and parse
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Extract match data
        matches = []
        match_elements = soup.find_all('div', class_='event__match')
        
        for match in match_elements:
            try:
                # Extract home team
                home_team_elem = match.find('div', class_='event__homeParticipant')
                home_team = home_team_elem.find('span', {'data-testid': 'wcl-scores-simple-text-01'}).text.strip() if home_team_elem else None
                
                # Extract away team  
                away_team_elem = match.find('div', class_='event__awayParticipant')
                away_team = away_team_elem.find('span', {'data-testid': 'wcl-scores-simple-text-01'}).text.strip() if away_team_elem else None
                
                # Extract scores
                home_score_elem = match.find('span', class_='event__score--home')
                home_score = home_score_elem.text.strip() if home_score_elem else None
                
                away_score_elem = match.find('span', class_='event__score--away') 
                away_score = away_score_elem.text.strip() if away_score_elem else None
                
                # Extract match date
                date_elem = match.find('div', class_='event__time')
                match_date = date_elem.text.strip() if date_elem else None
                
                # Extract match URL for more details
                link_elem = match.find('a', class_='eventRowLink')
                match_url = link_elem['href'] if link_elem else None
                
                if home_team and away_team:
                    matches.append({
                        'home_team': home_team,
                        'away_team': away_team,
                        'home_score': home_score,
                        'away_score': away_score,
                        'date': match_date,
                        'match_url': match_url,
                        'league': league_name
                    })
                    
            except Exception as e:
                print(f"  ✗ Error parsing match: {e}")
                continue
        
        print(f"  ✓ Scraped {len(matches)} matches from {league_name}")
        return matches
        
    except Exception as e:
        print(f"  ✗ Error scraping {league_name}: {e}")
        return []

# Initialize driver once for all leagues
driver = initialize_driver()
print("Chrome WebDriver initialized")

# Collect all matches from all leagues
all_matches = []
failed_leagues = []

try:
    for league_name, url in league_urls.items():
        try:
            league_matches = scrape_league_matches(driver, url, league_name)
            all_matches.extend(league_matches)
            time.sleep(2)  # Brief pause between leagues
            
        except Exception as e:
            print(f"Failed to scrape {league_name}: {e}")
            failed_leagues.append(league_name)
            continue

finally:
    # Close driver
    driver.quit()
    print("\nChrome WebDriver closed")

# Convert to DataFrame
df_all_leagues = pd.DataFrame(all_matches)

# Display summary
print(f"\n{'='*60}")
print("SCRAPING SUMMARY")
print(f"{'='*60}")
print(f"Total matches collected: {len(df_all_leagues)}")
print(f"Leagues successfully scraped: {len(league_urls) - len(failed_leagues)}")
print(f"Failed leagues: {failed_leagues if failed_leagues else 'None'}")

if not df_all_leagues.empty:
    print(f"\nMatches by league:")
    league_counts = df_all_leagues['league'].value_counts()
    for league, count in league_counts.items():
        print(f"  {league}: {count} matches")
    
    # Save to CSV
    save_dir = Path("../../data/raw")
    save_dir.mkdir(parents=True, exist_ok=True)
    output_path = save_dir / "match_results_all_leagues.csv"
    df_all_leagues.to_csv(output_path, index=False)
    print(f"\n✓ Data saved to: {output_path}")
    
    print(f"\nFirst few rows:")
    print(df_all_leagues.head())
else:
    print("No data collected")

Chrome WebDriver initialized

Scraping: Premier League 2022-2023
URL: https://www.flashscoreusa.com/soccer/england/premier-league-2022-2023/results/
  → Clicked 'Show more games' - Click #1
  → Clicked 'Show more games' - Click #2
  → Clicked 'Show more games' - Click #3
  ✓ No more 'Show more games' button found
  ✓ Loaded all games after 3 clicks
  ✓ Scraped 380 matches from Premier League 2022-2023

Scraping: Serie A 2022-2023
URL: https://www.flashscoreusa.com/soccer/italy/serie-a-2022-2023/results/
  → Clicked 'Show more games' - Click #1
  → Clicked 'Show more games' - Click #2
  → Clicked 'Show more games' - Click #3
  ✓ No more 'Show more games' button found
  ✓ Loaded all games after 3 clicks
  ✓ Scraped 381 matches from Serie A 2022-2023

Scraping: La Liga 2022-2023
URL: https://www.flashscoreusa.com/soccer/spain/laliga-2022-2023/results/
  → Clicked 'Show more games' - Click #1
  → Clicked 'Show more games' - Click #2
  → Clicked 'Show more games' - Click #3
  ✓ No more 'Sho

In [ ]:
df_all_leagues.to_csv("../../data/match_data_train/match_data.csv")